##### import

In [11]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm

# 파일 읽기
path_base = r"D:\3.자산\전산 dataset"
########################################################
company = "솔림"
workdate = "230926" # 폴더명
name_dataset = "개인회생새창_20230926_2217"
previous_workdate = "230904" # 폴더명
previous_name_dataset = "개인회생새창_20230904_1949_기준데이터" # 기준데이터 계좌키 미분리
########################################################
path_dataset = join(path_base, company, workdate)
path_previous = join(path_base, company, previous_workdate, previous_name_dataset+".xlsx")
path_result = join(path_dataset, name_dataset+"_기준데이터.xlsx")

##### 전산조회 전

In [12]:
previous = pd.read_excel(path_previous, sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})
path_raw_data = join(path_dataset, name_dataset+".xlsx")
raw_data_ori = pd.read_excel(path_raw_data, dtype={'채무자키':str, '분납키':str, '계좌키':str}).fillna("")

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [13]:
raw_data = raw_data_ori.copy()

# 컬럼 순서 바꾸기, 인트를 문자열로(타입때메 종료제거보다 나중에 나와야)
raw_data.insert(1, "분납키", raw_data.pop("분납키"))
raw_data.insert(7, "현재결과", raw_data.pop("현재결과"))
raw_data.insert(8, "분납상태", raw_data.pop("분납상태"))
raw_data.insert(9, "검수상태", raw_data.pop("검수상태"))


# 인가/미인가
인가미인가 = []
for i, v in tqdm(raw_data[["현재결과", "분납키", "총분납잔금", "현재원금"]].iterrows(), total=len(raw_data.index)) :
    result, opb = "", 0
    if v.현재결과 == "인가" : result = "인가"
    elif (v.현재결과 == "개시") or (v.현재결과 == "금지") : result = "미인가"
    elif (v.현재결과=="폐지취소결정") or (v.현재결과=="폐지취소결정송달") : 
        temp = previous.query('분납키==@v.분납키')["인가/미인가"].values
        if len(temp) == 1 : result = temp[0]
        else : result = "#N/A"
    else : result = "기타"

    인가미인가.append(result)


raw_data.insert(7, "인가/미인가", 인가미인가)
raw_data.insert(11, "반영OPB", None)

# 공채권
####### 통합신고채권(채권번호 하나)은 계좌별로 등록할 필요가 없으니 이렇게 적을 필요가 없음(아래서, 계좌통합건은 행분열하고 모두 0원으로 처리)
####### 통합신고채권은 계좌별로 분리하지 말고 통합해서 전산등록해야
raw_data.loc[raw_data.분납메모 == "공채권" , ["총분납금","분납미납금"]] = 0
raw_data.loc[raw_data.분납메모 == "공채권" , ["분납잔금","총분납잔금"]] = raw_data['총분납금'] - raw_data["총분납입금"]


# 엑셀 파일로 저장
if not os.path.exists(path_result):
    with pd.ExcelWriter(path_result, mode='w', engine='openpyxl') as writer:
        raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)
else : 
    with pd.ExcelWriter(path_result, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
        raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

100%|██████████| 9349/9349 [00:00<00:00, 16570.10it/s]


In [14]:
# 공채권 확인
raw_data.loc[raw_data.분납메모.str.match("(공|통합)채권"),:]

,채무자키,분납키,매각사구분,채권구분,분납자관계,분납자성명인,계좌키,인가/미인가,현재결과,분납상태,...,최종진행결과,사건키,자동최신화,최신화결과화면,관련최신화결과화면,공고게시일,공고물,공고변제율,공고변제기간,NOSER
2123,20410192,20105691,고려저축-01,IRL(인가),채무자,박승만,200974824,인가,인가,진행,...,,20378945,D,Y,,,,0,0,
3202,20421340,20146420,한울가람,NPL,채무자,금기정,200938541,인가,인가,진행,...,,2301027475,D,Y,,,,0,0,
3935,20436703,20125480,KB카드-01,NPL,채무자,오복희,200957789,인가,인가,진행,...,,20455053,D,Y,,,,0,0,
4543,20451606,20119907,고려저축-01,IRL(인가),채무자,정현철,200974148,인가,인가,진행,...,,20418662,D,Y,,,,0,0,
4593,20451815,20125237,고려저축-01,CCRS(개인),채무자,조영성,200974471,인가,인가,진행,...,,20453349,D,Y,,,,0,0,
4627,20451911,20119946,고려저축-01,IRL(미인가),채무자,지수홍,200974616,인가,인가,진행,...,,20418701,D,Y,,,,0,0,
4637,20451954,20119949,고려저축-01,IRL(인가),채무자,전순여,200974678,인가,인가,진행,...,,20418704,D,Y,,,,0,0,
4655,20451989,20119959,고려저축-01,IRL(인가),채무자,윤성중,200974745,인가,인가,진행,...,,20418714,D,Y,,,,0,0,
4720,20452190,20144370,고려저축-01,CCRS(개인),채무자,정민우,200975119,미인가,개시,진행,...,,2301013400,D,Y,,,,0,0,
4721,20452192,20119998,고려저축-01,IRL(인가),채무자,박경미,200975123,인가,인가,진행,...,,20418753,D,Y,,,,0,0,


##### 전산조회 할 거 작업 후
- 매각 제외 전이므로, 채무상태 및 담당자에서 종결, 매각건 제외
- 분납상태-[진행] & 
- 현재결과-[폐지취소결정, 폐지취소결정송달, 빈셀] & 
- 인가/미인가-[기타, #N/A] 즉 인가/미인가가 아닌 거

In [15]:
# 완성된 전체 다시 읽기
all_df = pd.read_excel(path_result, sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

# raw_data["계좌키"] = raw_data.분납키.apply(str) # 계좌키는 여러건 입력된 거 때문에 애초에 object타입이다.

# OPB
all_df['반영OPB'] = np.where(all_df['인가/미인가'] == '인가', all_df["총분납잔금"] , all_df["현재원금"])

# 진행중 1) --------------------
# 종료건 제외
if path_dataset.find("대성") < 0 : # 솔림
    ing_df = all_df[~(all_df.채무상태.str.match("종결|환매|매각")|all_df.담당자.str.match("종결|매각|매각예정|회계파산|DS|환매"))]
else : # 대성
    ing_df = all_df[~(all_df.채무상태.str.match("종결|환매|매각")|all_df.담당자.str.match("종결|매각|환매|파산|회계파산"))]
# 진행중
ing_df = ing_df[(ing_df["인가/미인가"] != "기타") & (ing_df["분납상태"] !="중단")]

###### 신복과 충돌하는 거 제거하기 #############################################################
if path_dataset.find("대성") < 0 : # 솔림
    개회제외 = {"채무자" : ["20462830"], "보증인" : [] } # 20462830 중복입금건. 신복이 기간이 더 남아서 신복으로 처리하기로 함
else : # 대성
    개회제외 = {"채무자" : [], "보증인" : [] } # 
##############################################################################################

# 진행중 2) -------------------- 매각건,파산건,신복중복건 포함 진행중
# # 종료건 제외
# if path_dataset.find("대성") < 0 : # 솔림
#     ing_df = all_df[~(all_df.채무상태.str.match("종결|환매")|all_df.담당자.str.match("종결|환매"))]
# else : # 대성
#     ing_df = all_df[~(all_df.채무상태.str.match("종결|환매")|all_df.담당자.str.match("종결|환매"))]
# # 진행중
# ing_df = ing_df[ing_df["분납상태"] !="중단"]

if len(개회제외["채무자"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["채무자"]) & (ing_df['분납자관계'] == '채무자'))]
if len(개회제외["보증인"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["보증인"]) & (ing_df['분납자관계'] == '보증인'))]

# 채무자, 보증인
debt_ing = ing_df[ing_df["분납자관계"]=="채무자"]
grt_ing = ing_df[ing_df["분납자관계"]=="보증인"]

# 저장
with pd.ExcelWriter(join(path_dataset, name_dataset+"_기준데이터.xlsx"), engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
    ing_df.to_excel(writer, sheet_name='개인회생(진행)', index=False)
    debt_ing.to_excel(writer, sheet_name='개인회생(진행)_채무자', index=False)
    grt_ing.to_excel(writer, sheet_name='개인회생(진행)_보증인', index=False)
    all_df.to_excel(writer, sheet_name='개인회생(전체)', index=False)

##### 계좌키 나누고 새로운 파일 저장
- 통합계좌 없으면 대성은 안 해도 됨

In [16]:
# 개인회생 계좌 나누기

# path_dataset = r"D:\3.자산\전산 dataset\230430(0501)"
# name_dataset = "개인회생새창_20230502_0850_기준데이터"
# ing_df = pd.read_excel(join(path_dataset, name_dataset+".xlsx"), sheet_name="개인회생(진행)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

split_ing_df = ing_df.copy()
split_ing_df["비고"] = ""
new_rows = []
for index, row in split_ing_df.iterrows():
    account_keys = row['계좌키'].split(',')
    first_account_key = account_keys.pop(0)
    split_ing_df.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

    for account_key in account_keys:
        new_row = row.copy() # 일단 모든 칼럼 복사
        new_row['계좌키'] = account_key
        new_row['반영OPB'],new_row['OPB'], new_row['총분납금'],new_row['총분납입금'],new_row['분납미납금'],new_row['분납잔금'],new_row['총분납잔금'] = 0,0,0,0,0,0,0 # 값 안 가져올 컬림은 빈문자열로 대체
        new_row["분납메모"] = "통합신고 : " + first_account_key
        new_rows.append(new_row)
        

temp = pd.DataFrame(new_rows)

result = pd.concat([split_ing_df, temp], ignore_index=True)

# 저장
writer = pd.ExcelWriter(join(path_dataset, name_dataset+"_진행중_계좌키분리_기준데이터.xlsx"))
result.to_excel(writer, sheet_name="개인회생(진행)", index=False)
writer.save()
writer.close()


C:\Users\SL\AppData\Local\Temp\ipykernel_29180\2913549892.py:30: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\sl\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
